In [1]:
import torch
from transformers import AutoTokenizer, AutoModel

MODEL_NAME = "Qwen/Qwen3-Embedding-0.6B"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

model = AutoModel.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    dtype=torch.float16,  # 🔑 FP16
    device_map="auto"           # 🔑 moves model to GPU
)

model.eval()

/Users/viriyadhika/Documents/5.UofT/Projects/SmallAgent/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Qwen3Model(
  (embed_tokens): Embedding(151669, 1024)
  (layers): ModuleList(
    (0-27): 28 x Qwen3DecoderLayer(
      (self_attn): Qwen3Attention(
        (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
        (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
        (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
        (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
      )
      (mlp): Qwen3MLP(
        (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
        (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
        (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
      (post_attention_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
    )
  )
  (norm): Qwen3RM

In [ ]:
from app.chat import MistralGenerator, DeepSeekGenerator, QwenGenerator
from app.common_agent import Router, Agent
from app.coding_agent import DataReaderTool, CodeWriter
from app.pdf_agent import PDFSplitter, PDFPartitioner, Embedder, ChromaStore, PDFIngestor, PDFIngestorTool, QueryAnswererTool, CrossEncoderReranker, RAGRetriever, PDFReaderTool, PDFSplitterTool

def remove_collection():
    temp_store = ChromaStore(path="./chroma_db", collection_name="rag")
    temp_store.client.delete_collection("rag")

def eval(question: str, max_steps: int):
    api_key = ""

    # Create your LLM wrapper
    # generator = MistralGenerator(api_key=api_key, temperature=0.1)
    
    generator_dev = MistralGenerator(api_key=api_key, temperature=0.1, model="mistralai/devstral-2512:free")

    generator = QwenGenerator(api_key=api_key)
    
    splitter = PDFSplitter(pages_per_part=20)
    partitioner = PDFPartitioner()
    embedder = Embedder(tokenizer, model, batch_size=8)

    store = ChromaStore(path="./chroma_db", collection_name="rag")

    reranker = CrossEncoderReranker(
        model_name="cross-encoder/ms-marco-MiniLM-L-6-v2",
        # model_name="cross-encoder/nli-deberta-v3-base",
        top_k=8
    )

    rag = RAGRetriever(
        embedder=embedder,
        vector_store=store,
        reranker=reranker,
        recall_k=20,
    )

    ingestor = PDFIngestor(
        splitter=splitter,
        partitioner=partitioner,
        embedder=embedder,
        store=store
    )

    # Register tools
    tools = [
        CodeWriter(generator_dev),
        DataReaderTool(),
        PDFIngestorTool(ingestor),
        QueryAnswererTool(rag, generator),
        PDFReaderTool(),
        PDFSplitterTool()
    ]

    # Create router / planner
    router = Router(generator, tools)


    # Create agent
    agent = Agent(
        router=router,
        tools=tools,
        max_steps=max_steps
    )

    return agent.run(question)


# 1. Simple Linear regression on data

In [5]:
eval(
    "Do a linear regression from data/data.csv and plot the curve fit, including residual plot and R^2 value",
    12
)

{'id': 'gen-1766855296-pljKxXlxBK5F8TYq5zoi', 'provider': 'Google', 'model': 'qwen/qwen3-235b-a22b-2507', 'object': 'chat.completion', 'created': 1766855296, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'STOP', 'index': 0, 'message': {'role': 'assistant', 'content': '{"action": "tool", "tool": "data_reader", "args": {"path": "data/data.csv"}}', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 778, 'completion_tokens': 25, 'total_tokens': 803, 'cost': 0.0002195, 'is_byok': False, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0, 'video_tokens': 0}, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.0001945, 'upstream_inference_completions_cost': 2.5e-05}, 'completion_tokens_details': {'reasoning_tokens': 0, 'image_tokens': 0}}}
Executing Tool: data_reader with Args {'path': 'data/data.csv'}
Tool Result: ToolResult(artifact_path='data/data_reader1.pkl', result={'info': "<class 'pandas.co

"Linear regression analysis has been completed using 'Zinc grade' as the predictor and 'Silver grade' as the response variable. The model achieved an R^2 value of 0.9906. The curve fit plot and residual plot have been generated and saved. All results, including the model, residuals, and plots, are available in the artifact."

# 2. Linear regression with dirty (with NaN) data

In [5]:
eval(
    "Do a linear regression from data/data_d.csv and plot the curve fit, including residual plot and R^2 value",
    12
)

{'id': 'gen-1766821623-aIhoShjRt9ggt2F3CoU1', 'provider': 'Novita', 'model': 'qwen/qwen3-235b-a22b-2507', 'object': 'chat.completion', 'created': 1766821623, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '{"action": "tool", "tool": "data_reader", "args": {"path": "data/data_d.csv"}}', 'refusal': None, 'reasoning': None}}], 'system_fingerprint': '', 'usage': {'prompt_tokens': 763, 'completion_tokens': 26, 'total_tokens': 789, 'cost': 6.7e-05, 'is_byok': False, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0, 'video_tokens': 0}, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 5.4936e-05, 'upstream_inference_completions_cost': 1.2064e-05}, 'completion_tokens_details': {'reasoning_tokens': 0, 'image_tokens': 0}}}
Executing Tool: data_reader with Args {'path': 'data/data_d.csv'}
Tool Result: ToolResult(artifact_path='data/data_reader1.pkl', re

"Linear regression has been performed using 'Zinc grade' as the predictor and 'Silver grade' as the response. The dataset was cleaned by removing rows with missing values in 'Silver grade'. The curve fit plot and residual plot have been generated and saved. The R^2 value of the model is included in the results. All outputs are available in the artifact."

# 3. Read an assignment in a PDF and CSV file and do it

In [6]:
eval(
    "Do the assignment from data/Assignment.pdf the data is available in data/data.csv",
    18
)

{'id': 'gen-1766821920-NlL7nzfj03e5cWSs9E7h', 'provider': 'WandB', 'model': 'qwen/qwen3-235b-a22b-2507', 'object': 'chat.completion', 'created': 1766821920, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '{"action": "tool", "tool": "pdf_read", "args": {"pdf_path": "data/Assignment.pdf"}}', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 756, 'completion_tokens': 27, 'total_tokens': 783, 'cost': 7.83e-05, 'is_byok': False, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0, 'video_tokens': 0}, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 7.56e-05, 'upstream_inference_completions_cost': 2.7e-06}, 'completion_tokens_details': {'reasoning_tokens': 0, 'image_tokens': 0}}}
Executing Tool: pdf_read with Args {'pdf_path': 'data/Assignment.pdf'}
Tool Result: ToolResult(artifact_path=None, result={'status': 'success', 'content': '[

"Assignment completed. For the weighted straight line fit (y = a + bx), the parameters are a = -1.715 and b = 2.975. The RMS error is 0.596 and R² is 0.993. For the weighted quadratic fit (y = a + bx + cx²), the parameters are a = -0.565, b = 1.719, and c = 0.235. The RMS error is 0.103 and R² is 0.9996. A plot comparing both models with the data (including error bars) has been saved as 'data/model_comparison.png'. Based on the lower RMS error and higher R², the quadratic model provides a better fit. The straight line model appears less accurate at higher zinc grades, potentially giving physically unrealistic predictions, whereas the quadratic model captures the curvature in the data more effectively."

# 4. Read the whole book and answer a question

In [6]:
eval(
    "Read the book in data/some_book.pdf and tell me who try to steal the stone?",
    12
)

{'id': 'gen-1766823699-8JGQPScwCOHla1JMrGLe', 'provider': 'Google', 'model': 'qwen/qwen3-235b-a22b-2507', 'object': 'chat.completion', 'created': 1766823699, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'STOP', 'index': 0, 'message': {'role': 'assistant', 'content': '{"action": "tool", "tool": "pdf_read", "args": {"pdf_path": "data/some_book.pdf"}}', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 775, 'completion_tokens': 28, 'total_tokens': 803, 'cost': 0.00022175, 'is_byok': False, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0, 'video_tokens': 0}, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.00019375, 'upstream_inference_completions_cost': 2.8e-05}, 'completion_tokens_details': {'reasoning_tokens': 0, 'image_tokens': 0}}}
Executing Tool: pdf_read with Args {'pdf_path': 'data/some_book.pdf'}
Tool Result: ToolResult(artifact_path=None, result={'status': 'failed', 'content': 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[Part 1/12] Ingested 26 chunks


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[Part 2/12] Ingested 31 chunks


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[Part 12/12] Ingested 1 chunks
Tool Result: ToolResult(artifact_path=None, result={'status': 'success', 'pdf_path': 'data/some_book.pdf', 'source': 'some_book'})
{'id': 'gen-1766824286-MvWUEUwyIvilm28UnOfG', 'provider': 'WandB', 'model': 'qwen/qwen3-235b-a22b-2507', 'object': 'chat.completion', 'created': 1766824286, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '{"action": "tool", "tool": "answer_question", "args": {"question": "Who tried to steal the stone in the book?"}}', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 5676, 'completion_tokens': 32, 'total_tokens': 5708, 'cost': 0.0005708, 'is_byok': False, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0, 'video_tokens': 0}, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.0005676, 'upstream_inference_completions_cost': 3.2e-06}, 'completion_tokens_details': {'reaso

Batches: 100%|██████████| 1/1 [00:00<00:00, 15.04it/s]


{'id': 'gen-1766824288-thWgmVE4sgit1inLwn63', 'provider': 'WandB', 'model': 'qwen/qwen3-235b-a22b-2507', 'object': 'chat.completion', 'created': 1766824288, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'The person who tried to steal the Stone in the book was **Professor Quirrell**.\n\nThis is supported by the context on **Page 213**, where Dumbledore says:  \n> “Professor Quirrell did not manage to take it from you. I arrived in time to prevent that…”  \n\nThis clearly indicates that Quirrell attempted to steal the Stone.', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 2808, 'completion_tokens': 76, 'total_tokens': 2884, 'cost': 0.0002884, 'is_byok': False, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0, 'video_tokens': 0}, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.0002808, 'upstream_inference_completions_cos

'Professor Quirrell tried to steal the stone.'

# 5. Longer Running coding Task

In [4]:
eval(
    "Load iris dataset from sklearn, Do a SVM and KNN comparison, visualize the confusion matrix and decision boundaries in PCA space, make sure all charts are labelled",
    12
)

{'id': 'gen-1766856659-T41muYaEPBPvYWsJptX8', 'provider': 'SiliconFlow', 'model': 'qwen/qwen3-235b-a22b-2507', 'object': 'chat.completion', 'created': 1766856659, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '{"action": "tool", "tool": "code_writer", "args": {"task": "Load iris dataset from sklearn and perform SVM classification", "context": "Use sklearn to load the iris dataset. Split the data into training and testing sets. Train an SVM classifier on the dataset. This is the first step toward comparing SVM and KNN models."}}', 'refusal': None, 'reasoning': None}}], 'system_fingerprint': '', 'usage': {'prompt_tokens': 787, 'completion_tokens': 73, 'total_tokens': 860, 'cost': 0.00011463, 'is_byok': False, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0, 'video_tokens': 0}, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 7.083e-05, 'upst

'The SVM and KNN models have been trained and evaluated on the iris dataset. Confusion matrices and decision boundaries in PCA-reduced 2D space have been successfully visualized. All charts are labeled with titles, axes, and legends. The comparison analysis is complete.'